In [2]:
!pip install albumentations

Looking in indexes: http://osrepo.barc.gov.in/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 10.9 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 11.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 9.6 MB/s eta 0:00:0031m12.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 9.8 MB/s eta 0:00:000m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 10.6 MB/s eta 0:00:00


In [14]:
import albumentations as A

import cv2
import os
os.environ['NO_VERSION_CHECK'] = '1'
import numpy as np

Augmenting Images - 600

In [ ]:
import cv2
import albumentations as A
import os
os.environ['NO_VERSION_CHECK'] = '1'
import numpy as np
import random

# Define the augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
   
    A.GridDistortion(p=0.3),
    
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.5),
    
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.1))

# Load image and bounding boxes from file
def load_image_and_boxes(image_path, label_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    boxes = []
    with open(label_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, w, h = map(float, line.strip().split())
            boxes.append([x_center, y_center, w, h, int(class_id)])
    return image, boxes

# Ensure bounding boxes fit within the image boundaries
def adjust_boxes(boxes):
    adjusted_boxes = []
    for box in boxes:
        x_center, y_center, w, h, class_id = box
        x_center = max(0, min(1, x_center))
        y_center = max(0, min(1, y_center))
        w = min(w, 1 - x_center)
        h = min(h, 1 - y_center)
        adjusted_boxes.append([x_center, y_center, w, h, class_id])
    return adjusted_boxes

# Clip bounding boxes to be within [0, 1]
def clip_boxes(boxes):
    clipped_boxes = []
    for bbox in boxes:
        x_center, y_center, w, h, class_id = bbox
        x_center = max(0, min(1, x_center))
        y_center = max(0, min(1, y_center))
        w = max(0, min(w, 1 - x_center))
        h = max(0, min(h, 1 - y_center))
        clipped_boxes.append([x_center, y_center, w, h, class_id])
    return clipped_boxes

# Apply augmentations and save the results
def augment_and_save(image, boxes, output_image_path, output_label_path, index):
    boxes = clip_boxes(boxes)  
    augmented = transform(image=image, bboxes=boxes)
    augmented_image = augmented['image']
    augmented_boxes = augmented['bboxes']
    
    adjusted_boxes = adjust_boxes(augmented_boxes)  
    
    output_image_filename = f"aug-{index}.png"  
    output_label_filename = f"aug-{index}.txt"  
    
    cv2.imwrite(os.path.join(output_image_path, output_image_filename), augmented_image)
    
    with open(os.path.join(output_label_path, output_label_filename), 'w') as file:
        for bbox in adjusted_boxes:
            x_center, y_center, w, h, class_id = bbox
            file.write(f'{class_id} {x_center} {y_center} {w} {h}\n')

# Paths to the input and output directories
input_images_dir = '/home/trainee/Desktop/Final_Work/YOLO-DATASET-WITH-AUG/images/train'
input_labels_dir = '/home/trainee/Desktop/Final_Work/YOLO-DATASET-WITH-AUG/labels/train'
output_images_dir = '/home/trainee/Desktop/Final_Work/YOLO-DATASET-WITH-AUG/aug_images_1'
output_labels_dir = '/home/trainee/Desktop/Final_Work/YOLO-DATASET-WITH-AUG/aug_labels_1'


os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Process each image and its corresponding label file
index = 1
processed_images = 0

for image_filename in os.listdir(input_images_dir):
    if image_filename.endswith('.png') and processed_images < 600:  
        image_path = os.path.join(input_images_dir, image_filename)
        label_filename = os.path.splitext(image_filename)[0] + '.txt'
        label_path = os.path.join(input_labels_dir, label_filename)
        
        if os.path.exists(label_path):
            image, boxes = load_image_and_boxes(image_path, label_path)
            
            # Check for presence of both class_id 0 and 1 in the image
            has_class_0 = any(box[4] == 0 for box in boxes)
            has_class_1 = any(box[4] == 1 for box in boxes)
            
            if has_class_0 and has_class_1:
                augment_and_save(image, boxes, output_images_dir, output_labels_dir, index)
                index += 1
                processed_images += 1

Visualisation of Augmented Images with corresponding Bounding Boxes

In [4]:

# Function to load image and bounding boxes from file
def load_image_and_boxes(image_path, label_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    boxes = []
    with open(label_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, w, h = map(float, line.strip().split())
            boxes.append([x_center, y_center, w, h, int(class_id)])
    return image, boxes

# Function to draw bounding boxes on the image
def draw_bboxes(image, boxes, output_visualization_path):
    height, width, _ = image.shape
    for box in boxes:
        x_center, y_center, w, h, class_id = box
        x_min = int((x_center - w / 2) * width)
        x_max = int((x_center + w / 2) * width)
        y_min = int((y_center - h / 2) * height)
        y_max = int((y_center + h / 2) * height)
        
        # Define colors based on class_id
        if class_id == 0:
            color = (255, 0, 0)  # Red color for class 0
        elif class_id == 1:
            color = (0, 255, 0)  # Green color for class 1
        else:
            color = (0, 0, 255)  # Blue color for other classes
        
        # Draw rectangle and class label
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
        cv2.putText(image, str(class_id), (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    
    cv2.imwrite(output_visualization_path, image)

# Paths to the input and output directories
input_images_dir = '/home/trainee/Desktop/Final_Work/YOLO-DATASET-WITH-AUG/aug_images_1'
input_labels_dir = '/home/trainee/Desktop/Final_Work/YOLO-DATASET-WITH-AUG/aug_labels_1'
output_visualization_dir = '/home/trainee/Desktop/Final_Work/YOLO-DATASET-WITH-AUG/aug_visualisation_1'

# Ensure output directory exists
os.makedirs(output_visualization_dir, exist_ok=True)

# Process each augmented image and its corresponding label file
for image_filename in os.listdir(input_images_dir):
    if image_filename.endswith('.png'):  # or any other image format
        image_path = os.path.join(input_images_dir, image_filename)
        label_filename = os.path.splitext(image_filename)[0] + '.txt'
        label_path = os.path.join(input_labels_dir, label_filename)
        
        if os.path.exists(label_path):
            # Load augmented image and adjusted bounding boxes
            image, boxes = load_image_and_boxes(image_path, label_path)
            
            # Create output visualization path
            output_visualization_path = os.path.join(output_visualization_dir, image_filename)
            
            # Draw bounding boxes on the augmented image and save visualization
            draw_bboxes(image, boxes, output_visualization_path)